# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment

from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import GridParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice 
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

from azureml.widgets import RunDetails

import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'og-azureml-capstone'

experiment = Experiment(ws, experiment_name)

In [ ]:
run = experiment.start_logging()

In [ ]:
# Create compute cluster
cluster_name = 'capstone-cluster'
try:
    cluster = ComputeTarget(workspace = ws, name = cluster_name)
    print("Cluster already exists, start using it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", max_nodes = 4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output = True)

In [ ]:
# Upload dataset to the workspace
data_path = "https://raw.githubusercontent.com/olgagnatenko13/nd00333-capstone/master/dataset/Dataset_for_Classification.csv"
found = False
key = "attrition-analysis"
description_text = "Employee attrition data"

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    dataset = Dataset.Tabular.from_delimited_files(data_path)        
      
    dataset = dataset.register(workspace=ws, name=key, description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling
# BanditPolicy allows to set up a relatively "aggressive" termination approach, and therefore allows to save VM resources
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation = 5)

#TODO: Create the different params that you will be using during training
# Grid sampling is used since both hyperparameters are selected from a finite list of values,
# and each value has its own impact on model performance
param_sampling = GridParameterSampling( {
    "--C": choice(0.01, 0.1, 1, 10, 100, 1000),
    "--gamma": choice(1, 0.1, 0.01, 0.001)
    }
)

NameError: name 'BanditPolicy' is not defined

In [ ]:
source_directory = "./training"
if "training" not in os.listdir():
    os.mkdir(source_directory)

In [ ]:
#TODO: Create your estimator and hyperdrive config
# since SKLearn estimator is deprecated, I use ScriptRunConfig

script_run_config = ScriptRunConfig(
    source_directory = source_directory,
    script = "train.py",
    compute_target = cluster,
    environment = environment
)

hyperdrive_run_config = HyperDriveConfig(
    run_config = script_run_config,
    hyperparameter_sampling = script_run_config,
    policy = early_termination_policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 30,
    max_duration_minutes = 60
)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = exp.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print("\nBest run metrics:", best_run_metrics)
print("\nParameter values:", parameter_values)

files = best_run.get_file_names()
best_model_name = files[-1]
best_run.download_file(best_model_name, output_file_path = "./outputs/")

best_model = joblib.load(best_model_name)
print("BEST MODEL", best_model_name, best_model)

In [ ]:
#TODO: Save the best model
output_folder='./outputs'
os.makedirs(output_folder, exist_ok=True)

joblib.dump(value=best_model, filename=best_model_name)

best_model = best_run.register_model(model_name = "best_hd_model", model_path = best_model_name, tags = best_run_metrics)
hd_accuracy = best_run_metrics["Accuracy"]
print("Accuracy: ", hd_accuracy)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service